# Лабораторна робота №7
Кіндратенко Ілля ФІТ 4-7
## Рекомендаційні системи
## Завдання 1
1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.
2. Виведіть перші 5 рядків завантаженого датасету.
3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.
5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).
6. Виведи рекомендації (10 фільмів) для конкретного користувача.

In [1]:
import pandas as pd

from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
from surprise import accuracy

In [4]:
data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'timestamp', 'timestamp'])

df.head()

,user_id,item_id,rate,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [5]:
# Визначте сітку параметрів для SVD
param_grid_svd = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

# Визначте сітку параметрів для KNNBasic
param_grid_knn = {
    'k': [20, 30, 40],
    'min_k': [1, 5, 10],
    'sim_options': {
        'name': ['msd', 'cosine'],
        'user_based': [False, True]
    }
}

# Виконайте GridSearchCV для SVD
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=3)
gs_svd.fit(data)

# Виконайте GridSearchCV для KNNBasic
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
gs_knn.fit(data)

# Виведіть найкращий результат і параметри для SVD
print("Best RMSE score for SVD: ", gs_svd.best_score['rmse'])
print("Best parameters for SVD: ", gs_svd.best_params['rmse'])

# Виведіть найкращий результат і параметри для KNNBasic
print("Best RMSE score for KNNBasic: ", gs_knn.best_score['rmse'])
print("Best parameters for KNNBasic: ", gs_knn.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matr

In [6]:
# Виведіть найкращу оцінку MAE та параметри для SVD
best_mae_svd = gs_svd.best_score['mae']
best_params_svd = gs_svd.best_params['mae']
print("Best MAE score for SVD: ", best_mae_svd)
print("Best parameters for SVD: ", best_params_svd)

# Виведіть найкращу оцінку MAE та параметри для KNNBasic
best_mae_knn = gs_knn.best_score['mae']
best_params_knn = gs_knn.best_params['mae']
print("Best MAE score for KNNBasic: ", best_mae_knn)
print("Best parameters for KNNBasic: ", best_params_knn)

# Виберіть найкращу модель на основі MAE
if best_mae_svd < best_mae_knn:
    best_model = 'SVD'
    best_score = best_mae_svd
    best_params = best_params_svd
else:
    best_model = 'KNNBasic'
    best_score = best_mae_knn
    best_params = best_params_knn

print(f"The best model is {best_model} with a MAE score of {best_score}")
print(f"Best parameters for the best model: {best_params}")

Best MAE score for SVD:  0.7339186528277675
Best parameters for SVD:  {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}
Best MAE score for KNNBasic:  0.779309747069572
Best parameters for KNNBasic:  {'k': 20, 'min_k': 1, 'sim_options': {'name': 'msd', 'user_based': True}}
The best model is SVD with a MAE score of 0.7339186528277675
Best parameters for the best model: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.05}


In [7]:
import random
trainset, testset = train_test_split(data, test_size=0.25)

# Навчіть найкращу модель
if best_model == 'SVD':
    algo = SVD(**best_params)
else:
    algo = KNNBasic(**best_params)

algo.fit(trainset)

# Виберіть випадкового користувача
user_id = random.choice(df['user_id'].unique())

# Отримати всі елементи, які користувач ще не оцінив
items = df['item_id'].unique()
rated_items = df[df['user_id'] == user_id]['item_id'].unique()
unrated_items = [item for item in items if item not in rated_items]

# Передбачення оцінок для елементів без оцінки
predictions = [algo.predict(user_id, item) for item in unrated_items]

# Отримайте 10 найкращих рекомендацій
top_10_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Виведіть рекомендації
print(f"Top 10 recommendations for user {user_id}:")
for prediction in top_10_recommendations:
    print(f"Item ID: {prediction.iid}, Predicted Rating: {prediction.est}")

Top 10 recommendations for user 359:
Item ID: 64, Predicted Rating: 5
Item ID: 12, Predicted Rating: 5
Item ID: 127, Predicted Rating: 4.953372359593025
Item ID: 192, Predicted Rating: 4.851766054309759
Item ID: 169, Predicted Rating: 4.836406883726329
Item ID: 302, Predicted Rating: 4.819680864255992
Item ID: 272, Predicted Rating: 4.8160895500289085
Item ID: 316, Predicted Rating: 4.774193263677531
Item ID: 511, Predicted Rating: 4.770451128282658
Item ID: 483, Predicted Rating: 4.769736720141254


## Висновок

У результаті використання бібліотеки Surprise для реалізації двох алгоритмів (SVD і KNNBasic) було виконано крос-валідацію для підбору оптимальних параметрів кожного алгоритму. Алгоритм SVD показав кращі результати на основі середньої абсолютної помилки (MAE), що робить його оптимальним для даного датасету. Після тренування на тренувальних даних, для випадкового користувача було згенеровано топ-10 рекомендацій фільмів, на основі передбачених рейтингів для нерейтингованих фільмів.

## Завдання 2
1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.
2. Отримайте
 рекомендацію
 для
 певного
 користувача.

In [8]:
from surprise import AlgoBase
from surprise import PredictionImpossible

class CustomAlgorithm(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)
    
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        return self
    
    def estimate(self, u, i):
        # Повертаємо середній рейтинг користувача
        if not self.trainset.knows_user(u) or not self.trainset.knows_item(i):
            raise PredictionImpossible('User or item is unknown.')
        
        user_ratings = [rating for (_, rating) in self.trainset.ur[u]]
        return sum(user_ratings) / len(user_ratings) if user_ratings else 3.0

# Створення екземпляра спеціального алгоритму
custom_algo = CustomAlgorithm()

# Навчання
custom_algo.fit(trainset)

# Передбачення рейтингів для набору тестів
custom_predictions = custom_algo.test(testset)

# Обчислити та надрукувати точність спеціального алгоритму
custom_rmse = accuracy.rmse(custom_predictions)
custom_mae = accuracy.mae(custom_predictions)

print(f"Custom Algorithm RMSE: {custom_rmse}")
print(f"Custom Algorithm MAE: {custom_mae}")

RMSE: 1.0471
MAE:  0.8398
Custom Algorithm RMSE: 1.0470898779815054
Custom Algorithm MAE: 0.8398462496820128


In [11]:
user_id = random.choice(df['user_id'].unique())

# Отримати всі елементи, які користувач ще не оцінив
items = df['item_id'].unique()
rated_items = df[df['user_id'] == user_id]['item_id'].unique()
unrated_items = [item for item in items if item not in rated_items]

# Прогнозуйте оцінки для елементів без рейтингу за допомогою 
predictions = [custom_algo.predict(user_id, item) for item in unrated_items]

# Отримайте 10 найкращих рекомендацій
top_10_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Вивід рекомендацій
print(f"Top 10 recommendations for user {user_id}:")
for prediction in top_10_recommendations:
    print(f"Item ID: {prediction.iid}, Predicted Rating: {prediction.est}")

Top 10 recommendations for user 839:
Item ID: 1492, Predicted Rating: 3.531013333333333
Item ID: 830, Predicted Rating: 3.531013333333333
Item ID: 1373, Predicted Rating: 3.531013333333333
Item ID: 857, Predicted Rating: 3.531013333333333
Item ID: 1374, Predicted Rating: 3.531013333333333
Item ID: 599, Predicted Rating: 3.531013333333333
Item ID: 1458, Predicted Rating: 3.531013333333333
Item ID: 1565, Predicted Rating: 3.531013333333333
Item ID: 1156, Predicted Rating: 3.531013333333333
Item ID: 1562, Predicted Rating: 3.531013333333333


## Висновок

Було побудовано власний алгоритм рекомендацій, який базується на середніх оцінках користувача для надання рекомендацій. Після тренування алгоритму, його точність була оцінена за допомогою RMSE та MAE, і були отримані рекомендації для конкретного користувача. Хоча цей алгоритм простіший за SVD і KNNBasic, він показує базову модель для побудови кастомної рекомендаційної системи.